# Rutinas para generación de insumos de límites flexibles y otros - Humedad relativa

> Elaborado por Paola Álvarez, profesional contratista IDEAM, contrato 196 de 2024. Comentarios o inquietudes, remitir a *palvarez@ideam.gov.co* 

**Librerías**

In [1]:
import pandas as pd
import numpy as np
import datetime
import statistics
import gc
import glob
import os
import csv
import re
import xarray as xr
import matplotlib.pyplot as plt
from openpyxl import Workbook
from openpyxl.chart import ScatterChart, Series, Reference
from openpyxl.utils.dataframe import dataframe_to_rows
#from openpyxl.chart.layout import Layout, ManualLayout
from collections import deque
from datetime import timedelta
from scipy import stats

____

### Se exportan coordenadas de HR

In [2]:
try:

    'hola'+ 5

except ValueError:

    print('Error 1')

except TypeError:

    print('Error 2')

except:

    print('Error 3')

Error 2


In [11]:
# Ruta a la carpeta con los archivos
carpeta = '../../../../2023/Obligaciones_Especificas/OE_2_ImplemCriteriosValidacion_EstAutom_Umbrales/DatosEMA_Crudos_Temp/Temp_Max_2m/RawTmax_Unmodified'

# Si la carpeta no existe, imprimir un mensaje
if not os.path.exists(carpeta):
    print(f"La carpeta '{carpeta}' no existe. Por favor, verificar la ruta.")

# Obtener la lista de todos los archivos en la carpeta
todos_los_archivos = os.listdir(carpeta)

# Filtrar los archivos que sigan el patrón 'Estacion_XXXXXXXXX.csv'
archivos = [f for f in todos_los_archivos if f.startswith('Estacion_') and len(f) == 23 and f[9:19].isdigit() and f.endswith('.csv')]

# Extraer los 10 últimos dígitos de cada nombre de archivo
componentes_numericos = [f[9:19] for f in archivos]

# Imprimir la lista resultante
print(componentes_numericos)

['0011030010', '0011035030', '0011045010', '0011050020', '0011080010', '0011105020', '0011115020', '0011120040', '0011135030', '0011159010', '0011175000', '0012015060', '0012015100', '0012015110', '0012025040', '0012045020', '0013035501', '0013055040', '0013085030', '0013085050', '0014015010', '0014015020', '0014015080', '0015015050', '0015015060', '0015015100', '0015015120', '0015035020', '0015065040', '0015065180', '0015065190', '0015065501', '0015075150', '0015075501', '0015079010', '0015085030', '0015085050', '0016015110', '0016015120', '0016015130', '0016015501', '0016025040', '0016035010', '0016035030', '0016040050', '0016055120', '0017015010', '0017025020', '0017035010', '0021015030', '0021015040', '0021015050', '0021015060', '0021015070', '0021025030', '0021035040', '0021045010', '0021050060', '0021050090', '0021055070', '0021065040', '0021095010', '0021105030', '0021105040', '0021105050', '0021115010', '0021115100', '0021115170', '0021115180', '0021125010', '0021130050', '0021

In [4]:
EMAHRcod = pd.DataFrame(componentes_numericos,columns=['Station'])
EMAHRcod

,Station
0,0011025501
1,0011030010
2,0011035030
3,0011045010
4,0011050020
...,...
615,5205500123
616,5311500056
617,5311500121
618,5311500147


In [5]:
EMAHRcod['Station'] = EMAHRcod['Station'].astype('int64')

In [6]:
EMAHRcod.dtypes

Station    int64
dtype: object

In [7]:
#dfsttnallinfo = pd.read_csv('stations_all.csv')
#dfsttnallinfo
CNE_All = pd.read_excel('../1_PresionAtmosferica/CNE_All_2024_07.xlsx')

In [8]:
CNE_All.rename(columns={'CODIGO':'Station','LATITUD':'lat','LONGITUD':'long', 'ENTIDAD':'entidad'}, inplace=True)

In [9]:
# Se unen ambos dataframe para obtener la información de coordenadas
dfsttninfo_EMAHR_Lctn = pd.merge(EMAHRcod, CNE_All[['Station','lat','long','entidad']], on='Station', how='left')
dfsttninfo_EMAHR_Lctn

,Station,lat,long,entidad
0,11025501,5.888719,-76.145167,CENTRO NACIONAL DE INVESTIGACIONES DE CAFE
1,11030010,5.380000,-76.610000,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...
2,11035030,5.284828,-76.627822,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...
3,11045010,5.690556,-76.643778,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...
4,11050020,5.994722,-76.780028,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...
...,...,...,...,...
615,5205500123,1.054806,-77.270250,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...
616,5311500056,3.848308,-76.987017,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...
617,5311500121,3.645194,-76.564750,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...
618,5311500147,3.890083,-77.070331,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...


In [10]:
dfsttninfo_EMAHR_Lctn.shape

(620, 4)

In [11]:
dfsttninfo_EMAHR_Lctn.head(2)

,Station,lat,long,entidad
0,11025501,5.888719,-76.145167,CENTRO NACIONAL DE INVESTIGACIONES DE CAFE
1,11030010,5.380000,-76.610000,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...


In [12]:
dfsttninfo_EMAHR_Lctn[dfsttninfo_EMAHR_Lctn.duplicated(subset=['Station'])]

,Station,lat,long,entidad


In [25]:
filtro = dfsttninfo_EMAHR_Lctn['lat'].isna()

In [26]:
dfsttninfo_EMAHR_Lctn[filtro]

,lat,long,entidad
Station,,,


In [16]:
dfsttnallinfo.set_index('Station', inplace=True)

In [22]:
dfsttninfo_EMAHR_Lctn.set_index('Station',inplace=True)

In [23]:
dfsttnallinfo_sbst = dfsttnallinfo[['lat', 'long']]
dfsttninfo_EMAHR_Lctn[['lat', 'long']] = dfsttninfo_EMAHR_Lctn[['lat', 'long']].fillna(dfsttnallinfo_sbst)

In [24]:
dfsttninfo_EMAHR_Lctn[dfsttninfo_EMAHR_Lctn.duplicated()]

,lat,long,entidad
Station,,,
21205514,4.469316,-74.079454,CAR - CORPORACION AUTONOMA REGIONAL DE CUNDINA...
21205523,4.788157,-74.050059,CAR - CORPORACION AUTONOMA REGIONAL DE CUNDINA...
21206790,5.090500,-73.881250,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...
25025350,9.316389,-75.387500,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...


In [27]:
dfsttninfo_EMAHR_Lctn.reset_index(inplace=True)

In [ ]:
dfsttninfo_EMAHR_Lctn[filtro]

In [29]:
dfsttninfo_EMAHR_Lctn.shape

(620, 4)

In [30]:
dfsttninfo_EMAHR_Lctn.to_excel('EMA_HR_coord.xlsx')

____ 

## Unir coords, freq, Ent con alturas, regiones naturales y regiones pluviométricas homogéneas (ACP)

In [2]:
freqInstData = pd.read_csv('EMAHR_LatLonEntFreq2.csv', encoding='latin-1')

In [26]:
freqInstData['entidad'].unique()

array(['CENTRO NACIONAL DE INVESTIGACIONES DE CAFE', 'IDEAM', 'INVEMAR',
       'INSTITUTO DE INVESTIGACIONES MARINAS Y COSTERAS JOSE BENITO VIVES DE ANDREIS',
       'CAR - CORPORACION AUTONOMA REGIONAL DE CUNDINAMARCA',
       'FONDO DE PREVENCION Y ATENCION DE DESASTRES DE BOGOTA DC - FOPAE',
       'ESTACIONES PARTICULARES',
       'EMPRESA DE ACUEDUCTO Y ALCANTARILLADO DE BOGOTA E.S.P',
       'ISAGEN  S.A. E.S.P', 'CORPORACION REGIONAL DEL QUINDIO',
       'INSTITUTO DISTRITAL DE GESTIÓN DE RIESGOS Y CAMBIO CLIMÁTICO'],
      dtype=object)

In [22]:
AltRegs= pd.read_table('EMAHR_GeoInfo_Replaceable.txt', encoding='latin-1', sep=';')

In [4]:
freqInstData.head(2)

,Unnamed: 0,Station,lat,long,entidad,FreqInf
0,0,11025501,5.888719,-76.145167,CENTRO NACIONAL DE INVESTIGACIONES DE CAFE,5T
1,1,11030010,5.380000,-76.610000,IDEAM,H


In [23]:
AltRegs['RegsNat'].unique()

array(['Andina', 'PacÃ\xadfico', 'Caribe', 'OrinoquÃ\xada',
       'AmazonÃ\xada', 'Insular'], dtype=object)

In [19]:
AltRegs.head(3)

,OBJECTID,F1,Station,lat,long_,entidad,RASTERVALU,RegsNat
0,1,0,11025501,5.888719,-76.145167,CENTRO NACIONAL DE INVESTIGACIONES DE CAFE,1649,Andina
1,2,1,11030010,5.380000,-76.610000,IDEAM,92,Pacífico
2,3,2,11035030,5.284828,-76.627822,IDEAM,102,Pacífico


In [20]:
dfMoreInfo = pd.merge(freqInstData, AltRegs[['Station','RASTERVALU','RegsNat']], on='Station', how='left')
dfMoreInfo.head(2)

,Unnamed: 0,Station,lat,long,entidad,FreqInf,RASTERVALU,RegsNat
0,0,11025501,5.888719,-76.145167,CENTRO NACIONAL DE INVESTIGACIONES DE CAFE,5T,1649.0,Andina
1,1,11030010,5.380000,-76.610000,IDEAM,H,92.0,Pacífico


In [27]:
dfMoreInfo.to_csv('EMAHR_Allinfo_Replcbl.csv', index=False,encoding='latin-1')

Se une la información de RP-ACP

In [2]:
dfAllInf = pd.read_csv('EMAHR_Allinfo_Replcbl.csv', encoding='latin-1', sep=';')
dfAllInf

,Unnamed: 0,Station,lat,long,entidad,FreqInf,Observ,RASTERVALU,RegsNat
0,0,11025501,5.888719,-76.145167,CENICAFE,5T,NaN,1649.0,Andina
1,1,11030010,5.380000,-76.610000,IDEAM,H,NaN,92.0,Pacífico
2,2,11035030,5.284828,-76.627822,IDEAM,H,NaN,102.0,Pacífico
3,3,11045010,5.690556,-76.643778,IDEAM,H,NaN,53.0,Pacífico
4,4,11050020,5.994722,-76.780028,IDEAM,H,NaN,25.0,Pacífico
...,...,...,...,...,...,...,...,...,...
615,615,5205500123,1.054806,-77.270250,IDEAM,H,NaN,2966.0,Andina
616,616,5311500056,3.848308,-76.987017,IDEAM,H,NaN,27.0,Pacífico
617,617,5311500121,3.645194,-76.564750,IDEAM,H,NaN,1637.0,Andina
618,618,5311500147,3.890083,-77.070331,IDEAM,NaN,NaN,6.0,Pacífico


In [24]:
RP_ACP = pd.read_table('EMAHR_Alt_RN-RPacp.txt', encoding='latin-1', sep=';')
RP_ACP

,OBJECTID,Station,lat,long_,entidad,RASTERVALU,RegsNat,NOMZH,ID_REG,LimSup
0,1,11025501,5.888719,-76.145167,CENTRO NACIONAL DE INVESTIGACIONES DE CAFE,1649,Andina,Medio Magdalena,11,313.95
1,2,11030010,5.380000,-76.610000,IDEAM,92,PacÃ­fico,PacÃ­fico - Directos,10,378.35
2,3,11035030,5.284828,-76.627822,IDEAM,102,PacÃ­fico,PacÃ­fico - Directos,10,378.35
3,4,11045010,5.690556,-76.643778,IDEAM,53,PacÃ­fico,PacÃ­fico - Directos,10,378.35
4,5,11050020,5.994722,-76.780028,IDEAM,25,PacÃ­fico,PacÃ­fico - Directos,10,378.35
...,...,...,...,...,...,...,...,...,...,...
609,610,5205500123,1.054806,-77.270250,IDEAM,2966,Andina,SaldaÃ±a,21,253.00
610,611,5311500056,3.848308,-76.987017,IDEAM,27,PacÃ­fico,Amarales - Dagua - Directos,16,399.05
611,612,5311500121,3.645194,-76.564750,IDEAM,1637,Andina,SaldaÃ±a,17,230.00
612,613,5311500147,3.890083,-77.070331,IDEAM,6,PacÃ­fico,Amarales - Dagua - Directos,16,399.05


In [25]:
dfwithACP = pd.merge(dfAllInf, RP_ACP[['Station','ID_REG']], on='Station', how='left')
dfwithACP

,Unnamed: 0,Station,lat,long,entidad,FreqInf,Observ,RASTERVALU,RegsNat,ID_REG
0,0,11025501,5.888719,-76.145167,CENICAFE,5T,NaN,1649.0,Andina,11.0
1,1,11030010,5.380000,-76.610000,IDEAM,H,NaN,92.0,Pacífico,10.0
2,2,11035030,5.284828,-76.627822,IDEAM,H,NaN,102.0,Pacífico,10.0
3,3,11045010,5.690556,-76.643778,IDEAM,H,NaN,53.0,Pacífico,10.0
4,4,11050020,5.994722,-76.780028,IDEAM,H,NaN,25.0,Pacífico,10.0
...,...,...,...,...,...,...,...,...,...,...
615,615,5205500123,1.054806,-77.270250,IDEAM,H,NaN,2966.0,Andina,21.0
616,616,5311500056,3.848308,-76.987017,IDEAM,H,NaN,27.0,Pacífico,16.0
617,617,5311500121,3.645194,-76.564750,IDEAM,H,NaN,1637.0,Andina,17.0
618,618,5311500147,3.890083,-77.070331,IDEAM,NaN,NaN,6.0,Pacífico,16.0


In [26]:
dfwithACP.drop(columns={'Unnamed: 0'}, inplace=True)

In [27]:
dfwithACP.rename(columns={'ID_REG':'RP_ACP'}, inplace=True)

In [29]:
dfwithACP.to_csv('EMAHR_Allinfo_Replcble.csv', index=False, encoding='latin-1')

In [30]:
groupsttn = dfwithACP.groupby('RP_ACP')

groupsttn['Station'].count()

RP_ACP
2.0     13
3.0     14
4.0     18
5.0     14
6.0     29
7.0     37
8.0     18
9.0      4
10.0    21
11.0    50
12.0    54
13.0    11
14.0    82
15.0    99
16.0     4
17.0    18
18.0    19
19.0    33
20.0     4
21.0    38
22.0    16
23.0    12
24.0     1
25.0     1
26.0     4
Name: Station, dtype: int64

______

## Cálculo de límites por región natural

In [18]:
def obtener_EstadDescript(archivo):
    try:
        datos = pd.read_csv(archivo, encoding='latin-1', dtype={'Valor': 'float32'})
        datos['Fecha'] = pd.to_datetime(datos['Fecha'], errors='coerce')
        datos = datos[(datos['Valor'] >= 2.0) & (datos['Valor'] <= 100.0)]

        station = datos['Station'].values[0]
        stats = datos['Valor'].agg({
            'minimo': 'min',
            'maximo': 'max',
            'media': 'mean',
            'mediana': 'median',
            'desvest': 'std',
            'varianza': 'var',
            'percentil_97': lambda x: x.quantile(0.97),
            'percentil_3': lambda x: x.quantile(0.03)
        })

        return station, datos['Valor'], stats
    
    except Exception as e:
        print(f"Error con el archivo {archivo}: {e}")
        return None, None, {}

def procesar_archivos(archivos, referencia_df, ruta_carpeta):
    datos_por_region = {}

    for archivo in archivos:
        ruta_archivo = os.path.join(ruta_carpeta, archivo)
        station, valores, stats = obtener_EstadDescript(ruta_archivo)

        if station and valores is not None:
            regnat_match = referencia_df.loc[referencia_df['Station'] == station, 'RegsNat']
            if not regnat_match.empty:
                regnat = regnat_match.values[0]
                if regnat not in datos_por_region:
                    datos_por_region[regnat] = {
                        'Valor': [],
                        'Station': set(),
                    }
                datos_por_region[regnat]['Valor'].extend(valores.tolist())
                datos_por_region[regnat]['Station'].add(str(station))
    
    gc.collect()  # Liberar memoria
    return datos_por_region

def main():
    ruta_carpeta = 'RawUnmodified_HR'
    ruta_referencia = 'EMAHR_Allinfo_Replcble.csv'

    referencia_df = pd.read_csv(ruta_referencia, encoding='latin-1')
    archivos = [f for f in os.listdir(ruta_carpeta) if f.endswith('.csv')]
    
    # Procesar todos los archivos de una vez
    datos_por_region = procesar_archivos(archivos, referencia_df, ruta_carpeta)

    resultados_por_region = []
    
    for regnat, datos in datos_por_region.items():
        valores_series = pd.Series(datos['Valor'])
        stats_region = valores_series.agg({
            'percentil_97': lambda x: x.quantile(0.97),
            'percentil_3': lambda x: x.quantile(0.03),
            'minimo': 'min',
            'maximo': 'max',
            'media': 'mean',
            'mediana': 'median',
            'desvest': 'std',
            'varianza': 'var'
        })

        # Cálculo de límites inferior y superior
        LimInf = stats_region['media'] - 4 * stats_region['desvest']
        LimSup = stats_region['media'] + 4 * stats_region['desvest']

        resultados_por_region.append([
            regnat,
            stats_region['percentil_97'],
            stats_region['percentil_3'],
            ','.join(datos['Station']),
            stats_region['minimo'],
            stats_region['maximo'],
            stats_region['media'],
            stats_region['mediana'],
            stats_region['desvest'],
            stats_region['varianza'],
            LimInf,
            LimSup
        ])

    resultados_df = pd.DataFrame(resultados_por_region, columns=[
        'RegNat', 
        'percentil_97', 
        'percentil_3', 
        'stations', 
        'minimo', 
        'maximo', 
        'media', 
        'mediana', 
        'desvest', 
        'varianza',
        'LimInf',
        'LimSup'
    ])
    
    print(resultados_df)
    resultados_df.to_csv('LimsHR_Estad.csv', index=False, encoding='latin-1')

if __name__ == "__main__":
    main()

Error con el archivo RawUnmodified_HR\Estacion_0021237020.csv: index 0 is out of bounds for axis 0 with size 0
Error con el archivo RawUnmodified_HR\Estacion_0024027070.csv: index 0 is out of bounds for axis 0 with size 0
Error con el archivo RawUnmodified_HR\Estacion_0032067030.csv: index 0 is out of bounds for axis 0 with size 0
Error con el archivo RawUnmodified_HR\Estacion_0054017040.csv: index 0 is out of bounds for axis 0 with size 0
Error con el archivo RawUnmodified_HR\Estacion_2120000099.csv: index 0 is out of bounds for axis 0 with size 0
Error con el archivo RawUnmodified_HR\Estacion_2120000100.csv: index 0 is out of bounds for axis 0 with size 0
      RegNat  percentil_97  percentil_3  \
0     Andina         100.0    41.860001   
1   Pacífico         100.0    59.000000   
2     Caribe         100.0    38.000000   
3        NaN         100.0     8.000000   
4  Orinoquía         100.0    40.899656   
5   Amazonía         100.0    51.000000   
6    Insular         100.0     8.

## Cálculo de límites por región pluviométrica homogénea - ACP / Límite inferior

In [72]:
# V1 - con lotes
def obtener_EstadDescript(archivo):
    try:
        datos = pd.read_csv(archivo, encoding='latin-1', dtype={'Valor': 'float32'})
        datos['Fecha'] = pd.to_datetime(datos['Fecha'], errors='coerce')
        datos = datos[(datos['Valor'] >= 0.0) & (datos['Valor'] <= 100.0)]

        station = datos['Station'].values[0]
        stats = datos['Valor'].agg({
            'minimo': 'min',
            'maximo': 'max',
            'media': 'mean',
            'mediana': 'median',
            'desvest': 'std',
            'varianza': 'var',
            'percentil_97': lambda x: x.quantile(0.97),
            'percentil_3': lambda x: x.quantile(0.03)
        })

        return station, datos['Valor'], stats
    
    except Exception as e:
        print(f"Error con el archivo {archivo}: {e}")
        return None, None, {}

def procesar_lote(archivos, referencia_df, ruta_carpeta):
    datos_por_region = {}

    for archivo in archivos:
        ruta_archivo = os.path.join(ruta_carpeta, archivo)
        station, valores, stats = obtener_EstadDescript(ruta_archivo)

        if station and valores is not None:
            rp_match = referencia_df.loc[referencia_df['Station'] == station, 'RP_ACP']
            if not rp_match.empty:
                rp = rp_match.values[0]
                if rp not in datos_por_region:
                    datos_por_region[rp] = {
                        'Valor': [],
                        'stations': set(),
                    }
                datos_por_region[rp]['Valor'].extend(valores.tolist())
                datos_por_region[rp]['stations'].add(str(station))
    
    gc.collect()  # Liberar memoria
    return datos_por_region

def main():
    ruta_carpeta = 'RawUnmodified_HR'
    ruta_referencia = 'EMAHR_Allinfo_Replcble.csv'

    referencia_df = pd.read_csv(ruta_referencia, encoding='latin-1')
    archivos = [f for f in os.listdir(ruta_carpeta) if f.endswith('.csv')]
    
    resultados_por_region = []
    lote_size = 50  # Tamaño del lote, ajustar según sea necesario

    for i in range(0, len(archivos), lote_size):
        lote_archivos = archivos[i:i + lote_size]
        datos_por_region = procesar_lote(lote_archivos, referencia_df, ruta_carpeta)

        for rp, datos in datos_por_region.items():
            valores_series = pd.Series(datos['Valor'])
            stats_region = valores_series.agg({
                'percentil_97': lambda x: x.quantile(0.97),
                'percentil_3': lambda x: x.quantile(0.03),
                'minimo': 'min',
                'maximo': 'max',
                'media': 'mean',
                'mediana': 'median',
                'desvest': 'std',
                'varianza': 'var'
            })

            # Cálculo de límites inferior y superior
            LimInf = stats_region['media'] - 4 * stats_region['desvest']
            LimSup = stats_region['media'] + 4 * stats_region['desvest']

            resultados_por_region.append([
                rp,
                stats_region['percentil_97'],
                stats_region['percentil_3'],
                ','.join(datos['stations']),
                stats_region['minimo'],
                stats_region['maximo'],
                stats_region['media'],
                stats_region['mediana'],
                stats_region['desvest'],
                stats_region['varianza'],
                LimInf,
                LimSup
            ])

        # Limpiar datos por región después de cada lote
        del datos_por_region
        gc.collect()  # Liberar memoria

    resultados_df = pd.DataFrame(resultados_por_region, columns=[
        'RP_ACP', 
        'percentil_97', 
        'percentil_3', 
        'stations', 
        'minimo', 
        'maximo', 
        'media', 
        'mediana', 
        'desvest', 
        'varianza',
        'LimInf',
        'LimSup'
        
    ])
    
    print(resultados_df)
    resultados_df.to_csv('LimsHR_sigma_RP-ACP.csv', index=False)

if __name__ == "__main__":
    main()

Error con el archivo RawUnmodified_HR\Estacion_2120000099.csv: index 0 is out of bounds for axis 0 with size 0
Error con el archivo RawUnmodified_HR\Estacion_2120000100.csv: index 0 is out of bounds for axis 0 with size 0
     RP_ACP  percentil_97  percentil_3  \
0      11.0          98.0    54.764017   
1      10.0         100.0     0.000000   
2       9.0         100.0     0.000000   
3       6.0         100.0     0.000000   
4       4.0         100.0     0.000000   
..      ...           ...          ...   
96     15.0         100.0    63.000000   
97     13.0         100.0    43.000000   
98     21.0         100.0    64.000000   
99     16.0         100.0    64.120003   
100    17.0         100.0    72.000000   

                                              stations     minimo  maximo  \
0                           11115020,11115501,11025501   0.524446   100.0   
1    11045010,11175000,11120040,11035030,11050020,1...   0.000000   100.0   
2                  11135030,11105020,11159

In [7]:
# V2 sin lotes
def obtener_EstadDescript(archivo):
    try:
        datos = pd.read_csv(archivo, encoding='latin-1', dtype={'Valor': 'float32'})
        datos['Fecha'] = pd.to_datetime(datos['Fecha'], errors='coerce')
        datos = datos[(datos['Valor'] >= 5.0) & (datos['Valor'] <= 100.0)]

        station = datos['Station'].values[0]
        stats = datos['Valor'].agg({
            'minimo': 'min',
            'maximo': 'max',
            'media': 'mean',
            'mediana': 'median',
            'desvest': 'std',
            'varianza': 'var',
            'percentil_97': lambda x: x.quantile(0.97),
            'percentil_3': lambda x: x.quantile(0.03)
        })

        return station, datos['Valor'], stats
    
    except Exception as e:
        print(f"Error con el archivo {archivo}: {e}")
        return None, None, {}

def procesar_archivos(archivos, referencia_df, ruta_carpeta):
    datos_por_region = {}

    for archivo in archivos:
        ruta_archivo = os.path.join(ruta_carpeta, archivo)
        station, valores, stats = obtener_EstadDescript(ruta_archivo)

        if station and valores is not None:
            rp_match = referencia_df.loc[referencia_df['Station'] == station, 'RP_ACP']
            if not rp_match.empty:
                rp = rp_match.values[0]
                if rp not in datos_por_region:
                    datos_por_region[rp] = {
                        'Valor': [],
                        'stations': set(),
                    }
                datos_por_region[rp]['Valor'].extend(valores.tolist())
                datos_por_region[rp]['stations'].add(str(station))
    
    gc.collect()  # Liberar memoria
    return datos_por_region

def main():
    ruta_carpeta = 'RawUnmodified_HR'
    ruta_referencia = 'EMAHR_Allinfo_Replcble.csv'

    referencia_df = pd.read_csv(ruta_referencia, encoding='latin-1')
    archivos = [f for f in os.listdir(ruta_carpeta) if f.endswith('.csv')]
    
    # Procesar todos los archivos de una vez
    datos_por_region = procesar_archivos(archivos, referencia_df, ruta_carpeta)

    resultados_por_region = []
    
    for rp, datos in datos_por_region.items():
        valores_series = pd.Series(datos['Valor'])
        stats_region = valores_series.agg({
            'percentil_97': lambda x: x.quantile(0.97),
            'percentil_3': lambda x: x.quantile(0.03),
            'minimo': 'min',
            'maximo': 'max',
            'media': 'mean',
            'mediana': 'median',
            'desvest': 'std',
            'varianza': 'var'
        })

        # Cálculo de límites inferior y superior
        LimInf = stats_region['media'] - 4 * stats_region['desvest']
        LimSup = stats_region['media'] + 4 * stats_region['desvest']

        resultados_por_region.append([
            rp,
            stats_region['percentil_97'],
            stats_region['percentil_3'],
            ','.join(datos['stations']),
            stats_region['minimo'],
            stats_region['maximo'],
            stats_region['media'],
            stats_region['mediana'],
            stats_region['desvest'],
            stats_region['varianza'],
            LimInf,
            LimSup
        ])

    resultados_df = pd.DataFrame(resultados_por_region, columns=[
        'RP_ACP', 
        'percentil_97', 
        'percentil_3', 
        'stations', 
        'minimo', 
        'maximo', 
        'media', 
        'mediana', 
        'desvest', 
        'varianza',
        'LimInf',
        'LimSup'
    ])
    
    print(resultados_df)
    resultados_df.to_csv('LimsHR_sigma_RP-ACP_6.csv', index=False)

if __name__ == "__main__":
    main()

Error con el archivo RawUnmodified_HR\Estacion_0021237020.csv: index 0 is out of bounds for axis 0 with size 0
Error con el archivo RawUnmodified_HR\Estacion_0024027070.csv: index 0 is out of bounds for axis 0 with size 0
Error con el archivo RawUnmodified_HR\Estacion_0032067030.csv: index 0 is out of bounds for axis 0 with size 0
Error con el archivo RawUnmodified_HR\Estacion_0035037100.csv: index 0 is out of bounds for axis 0 with size 0
Error con el archivo RawUnmodified_HR\Estacion_0054017040.csv: index 0 is out of bounds for axis 0 with size 0
Error con el archivo RawUnmodified_HR\Estacion_2120000099.csv: index 0 is out of bounds for axis 0 with size 0
Error con el archivo RawUnmodified_HR\Estacion_2120000100.csv: index 0 is out of bounds for axis 0 with size 0
    RP_ACP  percentil_97  percentil_3  \
0     11.0    100.000000    41.598855   
1     10.0    100.000000    59.000000   
2      9.0    100.000000    51.000000   
3      6.0    100.000000    42.000000   
4      4.0    100.

In [31]:
dfestadRP = pd.read_csv('LimsHR_sigma_RP-ACP_6.csv', sep=';')
dfestadRP.head(2)

,RP_ACP,percentil_97,percentil_3,stations,minimo,maximo,media,mediana,desvest,varianza,LimInf,LimSup
0,2.0,100.0,37.000000,"15,085,000,531,506,500,000,000,000,000,000,000...",6.000000,100,73.754065,76.0,17.518683,306.904240,3.679334,143.828795
1,3.0,100.0,44.806129,"150,151,001,501,506,000,000,000,000,000,000,00...",5.002789,100,77.965406,79.0,16.870365,284.609201,10.483948,145.446865


In [32]:
#dfestadRP[dfestadRP['stations']=='17035010']
dfestadRP[dfestadRP['RP_ACP'].isna()]

,RP_ACP,percentil_97,percentil_3,stations,minimo,maximo,media,mediana,desvest,varianza,LimInf,LimSup
25,NaN,100.0,68.0,17015010,9.0,100,80.730285,80.0,7.421249,55.074934,51.045290,110.415280
26,NaN,96.0,31.0,17017001,7.0,100,76.606233,82.0,16.838533,283.536190,9.252101,143.960364
27,NaN,100.0,74.0,17025020,54.0,100,89.083598,89.0,7.648024,58.492269,58.491502,119.675693
28,NaN,92.0,6.0,17035010,6.0,100,19.339415,12.0,21.555403,464.635409,-66.882198,105.561028
29,NaN,100.0,33.0,57015010,5.2,100,89.295268,92.0,14.361290,206.246644,31.850109,146.740428
30,NaN,55.0,25.9,88112901,5.4,100,42.821340,45.0,7.857306,61.737260,11.392115,74.250564


In [33]:
dfestadRP['RP_ACP'] = dfestadRP['RP_ACP'].fillna(dfestadRP['stations'])

In [34]:
dfestadRP['RP_ACP'] = dfestadRP['RP_ACP'].astype('float32')

In [35]:
# Se lee nuevamente el archivo del join previo que contiene datos de freqs, alt, RN y RP
dfwRP = pd.merge(dfwithACP, dfestadRP[['RP_ACP','LimInf']], on='RP_ACP', how='left')
dfwRP.head(2)

,Station,lat,long,entidad,FreqInf,Observ,RASTERVALU,RegsNat,RP_ACP,LimInf
0,11025501,5.888719,-76.145167,CENICAFE,5T,NaN,1649.0,Andina,11.0,7.150661
1,11030010,5.380000,-76.610000,IDEAM,H,NaN,92.0,Pacífico,10.0,36.582471


In [36]:
dfwRP['Station'].dtype

dtype('int64')

In [37]:
dfwRP['Station'] = dfwRP['Station'].astype('str')

In [38]:
dfwRP['RP_ACP'].value_counts()

RP_ACP
15.0    99
14.0    82
12.0    54
11.0    50
21.0    38
7.0     37
19.0    33
6.0     29
10.0    21
18.0    19
17.0    18
4.0     18
8.0     18
22.0    16
3.0     14
5.0     14
2.0     13
23.0    12
13.0    11
9.0      4
26.0     4
20.0     4
16.0     4
25.0     1
24.0     1
Name: count, dtype: int64

In [39]:
dfwRP[dfwRP['LimInf'].isna()]

,Station,lat,long,entidad,FreqInf,Observ,RASTERVALU,RegsNat,RP_ACP,LimInf
57,17015010,12.587849,-81.701117,IDEAM,H,NaN,NaN,NaN,NaN,NaN
58,17017001,12.599306,-81.689361,INVEMAR,H,NaN,NaN,NaN,NaN,NaN
59,17025020,13.359500,-81.357722,IDEAM,H,NaN,NaN,NaN,NaN,NaN
60,17035010,15.797500,-79.852222,IDEAM,H,NaN,NaN,NaN,NaN,NaN
519,57015010,4.096000,-81.608833,EST_PART,H,NaN,NaN,NaN,NaN,NaN
521,88112901,NaN,NaN,IDEAM,5T,NaN,NaN,NaN,NaN,NaN


Segunda parte del merge, tiene en cuenta los valores de NaN del RP_ACP

In [40]:
dfestadRP = dfestadRP.rename(columns={'stations':'Station'})

In [41]:
# Se lee nuevamente el archivo del join previo que contiene datos de freqs, alt, RN y RP
# Combina df1 y df2 basándose en la columna 'RP_ACP' (o la columna de referencia)
df_combined = dfwRP.merge(dfestadRP[['Station', 'LimInf']], on='Station', how='left', suffixes=('', '_new'))

# Rellena los valores vacíos en df1['LimInf'] con los valores correspondientes en df2['LimInf']
df_combined['LimInf'] = df_combined['LimInf'].combine_first(df_combined['LimInf_new'])

# Elimina la columna '_new' ya que ya fue usada para el relleno
df_combined.drop(columns=['LimInf_new'], inplace=True)

In [42]:
df_combined[df_combined['LimInf'].isna()]

,Station,lat,long,entidad,FreqInf,Observ,RASTERVALU,RegsNat,RP_ACP,LimInf


In [43]:
df_combined['LimInf'].unique()

array([  7.15066149,  36.58247136,  25.39885909,  15.46150028,
        17.34192196,  10.48394793,   3.67933417,  13.68987482,
        51.04528974,   9.25210146,  58.49150217, -66.88219796,
        23.03727746,   9.7706562 ,  12.47396368,   6.64329611,
        10.59585194,  14.37691386,  11.59431877,  -0.4819376 ,
         2.11427568,  16.01869045,  22.31509932,  24.76380332,
        15.28516932,   6.52548999,  15.88245078,   8.45339943,
        45.24954504,  31.85010926,  11.39211501])

In [44]:
df_combined = df_combined.rename(columns={'LimInf':'LimInfRP'})

In [45]:
df_combined.to_csv('EMAHR_Allinfo_Replcble1.csv', encoding='latin-1')

__________

## Resultados pruebas de persistencias

Se genera un solo archivo que reuna todas las persistencias calculadas

In [13]:
# Se lee carpeta actual
carpeta = os.getcwd()
# Lista los archivos en la carpeta
archivos = os.listdir(carpeta)
dataframes = []

# Procesa los archivos que comienzan con 'cant_persist_'
for archivo in archivos:
    if archivo.startswith('cant_persist_'):
        ruta_archivo = os.path.join(carpeta, archivo)
        
        # Lee el archivo CSV
        df = pd.read_csv(ruta_archivo, encoding='latin-1')
        df = df.drop(columns='Unnamed: 0', errors='ignore')  # Elimina la columna si existe
        
        # Renombra la columna 'Cantidad_0PER0' para incluir el nombre del archivo
        nomb_archivo = archivo[13:15]
        df = df.rename(columns={'Cantidad_0PER0': f'nshifts_{nomb_archivo}'})
        
        # Añade el DataFrame a la lista
        dataframes.append(df)

# Realiza el merge de todos los DataFrames basados en la columna 'Archivo'
# El primer DataFrame será la base para hacer el merge
df1 = dataframes[0]

for df in dataframes[1:]:
    # Para evitar columnas duplicadas, utiliza how='outer'
    merged_df_q = pd.merge(df1, df, on='Archivo', how='outer')
    df1 = merged_df_q

# Revisar df resultante
orden_olumnas = ['Archivo','nshifts_3_','nshifts_4_','nshifts_5_','nshifts_6_','nshifts_7_',
                 'nshifts_8_','nshifts_9_','nshifts_10','nshifts_11','nshifts_12','nshifts_13']
merged_df_q = merged_df_q.reindex(columns=orden_olumnas)
print(merged_df_q.head(2))
merged_df_q.to_csv('cant_persist_allshifts.csv', encoding='latin-1')

                      Archivo  nshifts_3_  nshifts_4_  nshifts_5_  nshifts_6_  \
0  Estacion_0011025501_qc.csv        4694        3100        2196        1657   
1  Estacion_0011030010_qc.csv       11968       10225        8502        6831   

   nshifts_7_  nshifts_8_  nshifts_9_  nshifts_10  nshifts_11  nshifts_12  \
0        1314        1089         916         778         674         594   
1        5232        3786        2616        1777        1221         868   

   nshifts_13  
0         528  
1         652  


Unir resultado tabla de persistencias con la que posee frecuencias

In [4]:
freqsEntInfo = pd.read_csv('EMAHR_Allinfo_Replcbl.csv', sep=';', encoding='latin-1')
freqsEntInfo

,Unnamed: 0,Station,lat,long,entidad,FreqInf,Observ,RASTERVALU,RegsNat
0,0,11025501,5.888719,-76.145167,CENICAFE,5T,NaN,1649.0,Andina
1,1,11030010,5.380000,-76.610000,IDEAM,H,NaN,92.0,Pacífico
2,2,11035030,5.284828,-76.627822,IDEAM,H,NaN,102.0,Pacífico
3,3,11045010,5.690556,-76.643778,IDEAM,H,NaN,53.0,Pacífico
4,4,11050020,5.994722,-76.780028,IDEAM,H,NaN,25.0,Pacífico
...,...,...,...,...,...,...,...,...,...
615,615,5205500123,1.054806,-77.270250,IDEAM,H,NaN,2966.0,Andina
616,616,5311500056,3.848308,-76.987017,IDEAM,H,NaN,27.0,Pacífico
617,617,5311500121,3.645194,-76.564750,IDEAM,H,NaN,1637.0,Andina
618,618,5311500147,3.890083,-77.070331,IDEAM,NaN,NaN,6.0,Pacífico


In [5]:
PersistInfo = pd.read_csv('cant_persist_allshifts.csv', encoding='latin-1', sep=';')
PersistInfo

,Unnamed: 0,Archivo,nshifts_3_,nshifts_4_,nshifts_5_,nshifts_6_,nshifts_7_,nshifts_8_,nshifts_9_,nshifts_10,nshifts_11,nshifts_12,nshifts_13
0,0,11025501,4694,3100,2196,1657,1314,1089,916,778,674,594,528
1,1,11030010,11968,10225,8502,6831,5232,3786,2616,1777,1221,868,652
2,2,11035030,4895,4197,3518,2871,2276,1746,1298,938,675,494,387
3,3,11045010,5870,4373,3014,2014,1294,808,482,279,178,119,90
4,4,11050020,4674,3878,3099,2371,1711,1167,759,501,346,262,214
...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,615,5205500123,980,560,244,100,39,11,2,0,0,0,0
616,616,5311500056,6673,5300,3951,2761,1795,1095,640,373,216,138,100
617,617,5311500121,4956,3515,2316,1425,805,428,216,121,70,45,34
618,618,5311500147,0,0,0,0,0,0,0,0,0,0,0


In [6]:
PersistInfo.rename(columns={'Archivo':'Station'},inplace=True)
PersistInfo.head(2)

,Unnamed: 0,Station,nshifts_3_,nshifts_4_,nshifts_5_,nshifts_6_,nshifts_7_,nshifts_8_,nshifts_9_,nshifts_10,nshifts_11,nshifts_12,nshifts_13
0,0,11025501,4694,3100,2196,1657,1314,1089,916,778,674,594,528
1,1,11030010,11968,10225,8502,6831,5232,3786,2616,1777,1221,868,652


In [7]:
persisFreqs = pd.merge(PersistInfo, freqsEntInfo[['Station','FreqInf']], on='Station')
persisFreqs

,Unnamed: 0,Station,nshifts_3_,nshifts_4_,nshifts_5_,nshifts_6_,nshifts_7_,nshifts_8_,nshifts_9_,nshifts_10,nshifts_11,nshifts_12,nshifts_13,FreqInf
0,0,11025501,4694,3100,2196,1657,1314,1089,916,778,674,594,528,5T
1,1,11030010,11968,10225,8502,6831,5232,3786,2616,1777,1221,868,652,H
2,2,11035030,4895,4197,3518,2871,2276,1746,1298,938,675,494,387,H
3,3,11045010,5870,4373,3014,2014,1294,808,482,279,178,119,90,H
4,4,11050020,4674,3878,3099,2371,1711,1167,759,501,346,262,214,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,615,5205500123,980,560,244,100,39,11,2,0,0,0,0,H
616,616,5311500056,6673,5300,3951,2761,1795,1095,640,373,216,138,100,H
617,617,5311500121,4956,3515,2316,1425,805,428,216,121,70,45,34,H
618,618,5311500147,0,0,0,0,0,0,0,0,0,0,0,NaN


In [8]:
filt1 = persisFreqs[(persisFreqs['FreqInf'] == 'H')] #& (persisFreqs['nshifts_13'])]

In [9]:
filt1

,Unnamed: 0,Station,nshifts_3_,nshifts_4_,nshifts_5_,nshifts_6_,nshifts_7_,nshifts_8_,nshifts_9_,nshifts_10,nshifts_11,nshifts_12,nshifts_13,FreqInf
1,1,11030010,11968,10225,8502,6831,5232,3786,2616,1777,1221,868,652,H
2,2,11035030,4895,4197,3518,2871,2276,1746,1298,938,675,494,387,H
3,3,11045010,5870,4373,3014,2014,1294,808,482,279,178,119,90,H
4,4,11050020,4674,3878,3099,2371,1711,1167,759,501,346,262,214,H
5,5,11080010,5801,4499,3274,2250,1424,852,487,276,159,97,71,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,613,5102500128,13116,11605,10125,8674,7248,5856,4574,3503,2630,1950,1469,H
614,614,5202500132,951,699,523,391,285,205,144,101,67,47,35,H
615,615,5205500123,980,560,244,100,39,11,2,0,0,0,0,H
616,616,5311500056,6673,5300,3951,2761,1795,1095,640,373,216,138,100,H


In [10]:
filt1['nshifts_13'].value_counts().count()

242

In [11]:
filt1['nshifts_13'].sort_values(ascending=False).ravel()

array([37735, 34889, 32212, 23834, 21667, 21602, 20171, 19967, 18801,
       18555, 18427, 17987, 17953, 17334, 16926, 16920, 16193, 16133,
       15806, 14059, 13798, 13686, 12896, 12657, 11961, 11557, 11349,
       11119, 10999, 10868, 10413,  9989,  8936,  8463,  8303,  8154,
        7828,  7799,  7783,  7737,  7625,  7584,  7473,  7120,  6862,
        6852,  6760,  6709,  6641,  6499,  6367,  6138,  6127,  6067,
        6017,  5844,  5643,  5524,  5454,  5405,  5308,  5292,  5165,
        5093,  4983,  4861,  4817,  4740,  4632,  4608,  4394,  4347,
        4299,  4238,  4135,  4073,  3988,  3965,  3861,  3761,  3733,
        3603,  3503,  3462,  3228,  3059,  2967,  2720,  2606,  2558,
        2274,  2261,  2242,  2181,  2117,  2111,  2067,  2021,  1937,
        1916,  1833,  1817,  1778,  1732,  1717,  1716,  1712,  1670,
        1641,  1641,  1594,  1554,  1539,  1533,  1518,  1469,  1332,
        1289,  1276,  1234,  1223,  1202,  1167,  1140,  1120,  1043,
        1000,   997,

In [12]:
filt1['nshifts_13'].sort_values(ascending=False)

490    37735
55     34889
470    32212
311    23834
357    21667
       ...  
450        0
147        0
454        0
146        0
103        0
Name: nshifts_13, Length: 449, dtype: int64

In [13]:
persisFreqs.iloc[357]

Unnamed: 0         357
Station       26157170
nshifts_3_       33767
nshifts_4_       31756
nshifts_5_       30068
nshifts_6_       28596
nshifts_7_       27277
nshifts_8_       26073
nshifts_9_       24997
nshifts_10       24047
nshifts_11       23178
nshifts_12       22384
nshifts_13       21667
FreqInf              H
Name: 357, dtype: object

In [16]:
filt1[filt1['nshifts_13'] == 13]

,Unnamed: 0,Station,nshifts_3_,nshifts_4_,nshifts_5_,nshifts_6_,nshifts_7_,nshifts_8_,nshifts_9_,nshifts_10,nshifts_11,nshifts_12,nshifts_13,FreqInf
208,208,23065505,1167,687,389,212,126,76,51,36,27,19,13,H
612,612,3706500109,1722,1091,692,440,278,179,108,65,39,24,13,H


In [18]:
precipECM_R = pd.read_csv(r'C:\Users\user\Documents\IDEAM\2023\Obligaciones_Especificas\OE_2_ImplemCriteriosValidacion_EstAutom_Umbrales\DatosEMA_Crudos_Precip\1series_prpc_day_raw_melted.csv')

In [19]:
precipECM_R.head(2)

,Unnamed: 0,fecha,Station,Rainfall
0,0,1981-01-01,11010010,0.0
1,1,1981-01-02,11010010,0.0


In [39]:
filtfechas = precipECM_R[(precipECM_R['Station']==37055010)] #& (precipECM_R['fecha']=='2019-01')] # Se cambia para pruebas

In [40]:
filtfechas.set_index('fecha', inplace=True)
filtfechas

,Unnamed: 0,Station,Rainfall
fecha,,,
1981-01-01,18393990,37055010,0.0
1981-01-02,18393991,37055010,0.0
1981-01-03,18393992,37055010,0.0
1981-01-04,18393993,37055010,0.0
1981-01-05,18393994,37055010,0.0
...,...,...,...
2020-12-27,18408595,37055010,NaN
2020-12-28,18408596,37055010,NaN
2020-12-29,18408597,37055010,NaN


In [49]:
filtfechas['2018-11':'2019-02']['Rainfall'].ravel()

array([ 0. ,  0.2,  0. ,  0. ,  0. ,  0. , 13.4,  1. , 15.3,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  7.1, 16.5,  0.1,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  3.8,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. , 19.9,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ])

In [54]:
filtfechas['2018-11-01':'2018-11-09']

,Unnamed: 0,Station,Rainfall
fecha,,,
2018-11-01,18407808,37055010,0.0
2018-11-02,18407809,37055010,0.2
2018-11-03,18407810,37055010,0.0
2018-11-04,18407811,37055010,0.0
2018-11-05,18407812,37055010,0.0
2018-11-06,18407813,37055010,0.0
2018-11-07,18407814,37055010,13.4
2018-11-08,18407815,37055010,1.0
2018-11-09,18407816,37055010,15.3


------

In [75]:
filt1['nshifts_6_'].sort_values(ascending=False).ravel()

array([47875, 38329, 36582, 29657, 28965, 28596, 26544, 26106, 25528,
       25466, 23648, 21609, 21084, 19182, 19139, 19045, 18846, 18634,
       18238, 18086, 17632, 17275, 16825, 16457, 16223, 15155, 15147,
       14487, 14281, 13085, 12414, 11953, 11708, 11607, 11435, 10784,
       10715, 10668, 10511, 10102,  9980,  9906,  9778,  9473,  9461,
        9449,  9378,  9352,  9111,  9077,  8909,  8674,  8574,  8420,
        8166,  8037,  7912,  7892,  7785,  7765,  7710,  7507,  7442,
        7413,  6913,  6893,  6831,  6636,  6607,  6535,  6471,  6260,
        5987,  5884,  5643,  5628,  5574,  5556,  5502,  5487,  5366,
        5351,  5325,  5289,  5106,  5019,  4925,  4878,  4712,  4600,
        4597,  4503,  4402,  4364,  4350,  4340,  4280,  4280,  3972,
        3951,  3810,  3742,  3732,  3641,  3514,  3495,  3457,  3450,
        3284,  3271,  3218,  3076,  2992,  2952,  2916,  2871,  2857,
        2844,  2826,  2765,  2761,  2695,  2642,  2595,  2569,  2425,
        2416,  2377,

In [63]:
filt1[filt1['nshifts_3_'] == 0]

,Unnamed: 0,Station,nshifts_3_,nshifts_4_,nshifts_5_,nshifts_6_,nshifts_7_,nshifts_8_,nshifts_9_,nshifts_10,nshifts_11,nshifts_12,nshifts_13,FreqInf
11,11,11155030,0,0,150,119,94,73,55,42,32,25,19,H
71,71,21045010,0,0,0,0,0,0,0,0,0,0,0,H
82,82,21095010,0,0,0,0,0,0,0,0,0,0,0,H
88,88,21115100,0,0,0,0,0,0,0,0,0,0,0,H
178,178,21255090,0,0,0,0,0,0,0,0,0,0,0,H
192,192,23020080,0,0,0,0,0,0,0,0,0,0,0,H
250,250,24015513,0,0,0,0,0,0,0,0,0,0,0,H
365,365,26185020,0,0,0,0,0,0,0,0,0,0,0,H
402,402,29045180,0,0,0,0,0,0,0,0,0,0,0,H


----

In [87]:
filt2 = persisFreqs[(persisFreqs['FreqInf'] == '5T')]

In [88]:
filt2['nshifts_13'].sort_values(ascending=False).ravel()

array([22799, 10456,  9937,  9162,  8948,  7921,  7756,  7559,  7324,
        6585,  6326,  6188,  6088,  5959,  5949,  5899,  5598,  5491,
        4469,  4368,  4207,  4180,  3904,  3333,  3039,  3012,  2886,
        2722,  2643,  2604,  2446,  2413,  2302,  2293,  2135,  2077,
        2041,  1980,  1862,  1772,  1652,  1636,  1635,  1632,  1557,
        1472,  1326,  1298,  1270,  1181,  1156,  1109,  1103,  1089,
        1066,  1012,  1005,   970,   950,   935,   923,   919,   891,
         891,   831,   759,   722,   714,   712,   694,   681,   680,
         667,   656,   654,   607,   597,   569,   543,   531,   528,
         465,   464,   434,   423,   392,   392,   334,   330,   280,
         260,   230,   162,   136,   123,    74,    31,    30,     7,
           7,     0], dtype=int64)

In [95]:
filt2[filt2['nshifts_13'] == 423]

,Unnamed: 0,Station,nshifts_3_,nshifts_4_,nshifts_5_,nshifts_6_,nshifts_7_,nshifts_8_,nshifts_9_,nshifts_10,nshifts_11,nshifts_12,nshifts_13,FreqInf
194,194,23025501,3288,2050,1419,1075,873,731,633,559,505,460,423,5T


In [82]:
filtfechas2 = precipECM_R[(precipECM_R['Station']==23055501)]

In [84]:
filtfechas2

,Unnamed: 0,Station,Rainfall
fecha,,,


In [83]:
filtfechas2.set_index('fecha', inplace=True)
filtfechas2

,Unnamed: 0,Station,Rainfall
fecha,,,


In [ ]:
filtfechas2['2019-04':'2019-05']['Rainfall'].ravel()